## 1 Import libraries or packges that we need

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from model import mf, mfDataset

## 2 Set the training configuration, i.e. hyper-paramters, computing device, dataset path

In [ ]:
file_path = '../data/1m.dat'
batch_size = 2048
device = torch.device('cuda:0')
learning_rate = 1e-4
weight_decay = 3e-5
epochs = 100

## 3 Preprocessing before training

In [5]:
df = pd.read_csv(file_path, header=None, delimiter='::')
x, y = df.iloc[:, :2], df.iloc[:, 2]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

train_dataset = mfDataset(np.array(x_train[0]), np.array(
        x_train[1]),  np.array(y_train).astype(np.float32))
test_dataset = mfDataset(np.array(x_test[0]), np.array(
        x_test[1]), np.array(y_test).astype(np.float32))

train_DataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_DataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

mean_rating = df.iloc[:, 2].mean()
num_users = max(df[0])+1
num_items = max(df[1])+1
print(f"num_users:{num_users}")
print(f"num_items:{num_items}")

model = mf(num_users, num_items, mean_rating).to(device)
model_my_SGD=mf(num_users, num_items, mean_rating).to(device)

optimizer = torch.optim.SGD(
        params=model.parameters(), lr=learning_rate)

# optimizer = torch.optim.Adam(``
#         params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)

loss_func = torch.nn.MSELoss().to(device)

num_movies: 3883
num_users: 6040


## 4 Training and evaluation
optimization method: Stochastic Gradient Descent (SGD) or Adam

In [ ]:
for epoch in range(epochs):
    # training phase
    model.train()
    total_loss, total_len = 0, 0
    for x_u, x_i, y in train_DataLoader:
        x_u, x_i, y = x_u.to(device), x_i.to(device), y.to(device)
        y_pre = model(x_u, x_i)
        loss = loss_func(y_pre, y)

        # auto gradient computing and gradient descent based on pytorch
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()*len(y)
        total_len += len(y)
    train_loss = total_loss/total_len

    # evaluation phase
    model.eval()
    labels, predicts = [], []
    with torch.no_grad():
        for x_u, x_i, y in test_DataLoader:
            x_u, x_i, y = x_u.to(device), x_i.to(device), y.to(device)
            y_pre = model(x_u, x_i)
            labels.extend(y.tolist())
            predicts.extend(y_pre.tolist())
    mse = mean_squared_error(np.array(labels), np.array(predicts))

    print("epoch {}, train loss is {}, val mse is {}".format(
        epoch, train_loss, mse))